# Image recognition notebook

In [1]:
import tensorflow as tf
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.decomposition import PCA

# The "x" of the dataset contains a two dimension numpy array of uint8, where each
# row contains a 32x32 coloured image. The picture follows the "channel first" rule.

# The "y" of the dataset contains a one dimension numpy array of uint8, where each
# value indicates the label of corresponding x item.


# Load the recycled dataset
train_data = np.load('./dataset/recycled_32_train.npz')
test_data = np.load('./dataset/recycled_32_test.npz')

x_train = train_data['x']
y_train = train_data['y']
x_test = test_data['x']
y_test = test_data['y']

# Preprocess the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape the data
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)

x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

# Use quantum computing for the image classification model

# Define the quantum device
dev = qml.device("default.qubit", wires=4)

# Define the quantum circuit
@qml.qnode(dev)
def circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(4))
    qml.templates.BasicEntanglerLayers(weights, wires=range(4))
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]


# Define the quantum model
def quantum_model(inputs, weights):
    outputs = [circuit(inputs, w) for w in weights]
    return outputs




In [7]:
import warnings

import logging

logging.disable(logging.WARNING)
warnings.filterwarnings("ignore", message="You are casting an input of type complex128 to an incompatible dtype float32.")

In [4]:
# Step 1: Hybrid Quantum-Classical Model

# 1.1 Classical part: PCA and data flattening
n_components = 4
pca = PCA(n_components=n_components)
x_train_pca = pca.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_pca = pca.transform(x_test.reshape(x_test.shape[0], -1))

# 1.2 Quantum part: Quantum model
n_layers = 2  # Example: 2 layers for the BasicEntanglerLayers
weight_shapes = {"weights": (n_layers, 4)}

# Convert the quantum model to a Keras layer
quantum_layer = qml.qnn.KerasLayer(circuit, weight_shapes, output_dim=4)

x_train_pca = x_train_pca.astype(np.float32)
x_test_pca = x_test_pca.astype(np.float32)




# 1.3 Classical part: Define the full model using Keras
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(4, activation="relu"),  # for the reduced dimensionality from PCA
    quantum_layer,
    tf.keras.layers.Dense(6, activation="softmax")  # 6 classes as per the data provided
])


In [5]:
# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [8]:
# Step 2: Training
model.fit(x_train_pca, y_train, epochs=5, batch_size=32, validation_data=(x_test_pca, y_test))

Epoch 1/5
313/313 [==============================] - 22s 70ms/step - loss: 1.6987 - accuracy: 0.2639 - val_loss: 1.6025 - val_accuracy: 0.3073
Epoch 2/5
313/313 [==============================] - 22s 69ms/step - loss: 1.5453 - accuracy: 0.3481 - val_loss: 1.4920 - val_accuracy: 0.3787
Epoch 3/5
313/313 [==============================] - 22s 69ms/step - loss: 1.4655 - accuracy: 0.3736 - val_loss: 1.4421 - val_accuracy: 0.3867
Epoch 4/5
313/313 [==============================] - 22s 69ms/step - loss: 1.4312 - accuracy: 0.3810 - val_loss: 1.4170 - val_accuracy: 0.3927
Epoch 5/5
313/313 [==============================] - 22s 69ms/step - loss: 1.4141 - accuracy: 0.3858 - val_loss: 1.4024 - val_accuracy: 0.3867


In [9]:
# Step 3: Evaluation
loss, accuracy = model.evaluate(x_test_pca, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

47/47 [==============================] - 2s 32ms/step - loss: 1.4024 - accuracy: 0.3867
Loss: 1.4023897647857666
Accuracy: 0.3866666555404663
